In [ ]:
import gym
env = gym.make("LunarLander-v2")
import gym.wrappers

import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
BATCH_SIZE = 100
SESSION_SIZE = 500
PERCENTILE = 80
LEARNING_RATE = 0.0025
COMPLETION_SCORE = 200